In [1]:
from lion_service import iModel
from pydantic import BaseModel, Field
from lion_core.session.branch import Branch
from IPython.display import Markdown, display


gpt4omini = iModel(
    provider="openai",
    api_key="OPENAI_API_KEY",
    task="chat",
    model="gpt-4o-mini",
    interval_tokens=1_000_000,
    interval_requests=10_000,
    max_tokens=4_000,
    temperature=0.1,
)

gpt4o = iModel(
    provider="openai",
    api_key="OPENAI_API_KEY",
    task="chat",
    model="gpt-4o",
    interval_tokens=1_000_000,
    interval_requests=10_000,
    max_tokens=4_000,
    temperature=0.1,
)


class SelectionModel(BaseModel):
    selected: list = Field(
        [], description="select from [positive, negative, neutral]"
    )


async def mock_search(query: str):
    """
    A mock search function that returns a string
    """
    return f"Search results for: {query}"


branch = Branch(
    operative_model=SelectionModel,
    imodel=gpt4omini,
)

In [2]:
from lionfuncs import as_readable_json

a = await branch.operate(
    instruction="Answer the philosophical question provided",
    guidance="Reply concisely and informatively",
    context=["What is the meaning of life?"],
    reason=True,
    temperature=0.3,
    num_parse_retries=3,
)
print(as_readable_json(a))

{
    "title": "The Meaning of Life",
    "content": "The meaning of life is a subjective concept that varies from person to person. It can encompass the pursuit of happiness, fulfillment, relationships, personal growth, and contributing to society. Philosophers, theologians, and scientists have all offered different interpretations, suggesting that meaning may be derived from experiences, connections, and the quest for knowledge.",
    "reason": {
        "title": "Subjectivity of Meaning",
        "content": "The answer reflects the diverse perspectives on life's meaning, acknowledging that it is influenced by individual beliefs, experiences, and cultural contexts.",
        "confidence_score": 0.85
    },
    "action_requests": [],
    "action_required": false
}


In [3]:
branch.to_df()

,timestamp,lion_class,role,content,ln_id,sender,recipient,metadata
0,2024-10-21T13:19:32.843436,Instruction,user,{'guidance': 'Reply concisely and informativel...,ln784b1796-ae50da154-44f10fa3-c6-e7948232d,user,ln50d5f9d2ffd2-0389c-14d-1c9978f-936b71356,{'last_updated': {'recipient': 1729531175.2489...
1,2024-10-21T13:19:35.248644,AssistantResponse,assistant,"{'assistant_response': '```json { ""title"": ""...",lnd1221f9-6dba4d-863efd0-e59558-eee7663cd7,ln50d5f9d2ffd2-0389c-14d-1c9978f-936b71356,user,{'model_response': {'id': 'chatcmpl-AKqM1BiAl5...


In [4]:
res = await branch.operate(
    instruction="Use search tool to gather information on the topic, we are testing",
    guidance="Provide a summary of the information",
    context=["What is the meaning of life?"],
    reason=True,
    actions=True,
    tools=[mock_search],
    clear_messages=True,
    num_parse_retries=3,
)
Markdown(f"```json\n{as_readable_json(res)}\n```")

```json
{
    "title": "Exploring the Meaning of Life",
    "content": "The meaning of life is a philosophical question that has been contemplated by humans for centuries. Various perspectives exist, including religious, existential, and scientific viewpoints. Religions often provide a framework for understanding life's purpose, suggesting that it involves serving a higher power or achieving spiritual enlightenment. Existentialists argue that life has no inherent meaning, and it is up to individuals to create their own purpose through choices and actions. Scientific perspectives may focus on biological imperatives, such as survival and reproduction, as fundamental to understanding life. Ultimately, the meaning of life may vary greatly from person to person, influenced by personal beliefs, experiences, and cultural backgrounds.",
    "reason": {
        "title": "Comprehensive Overview",
        "content": "The response covers multiple perspectives on the meaning of life, providing a balanced view that acknowledges the complexity of the topic.",
        "confidence_score": 0.9
    },
    "action_requests": [],
    "action_required": false
}
```

In [5]:
branch.to_df()

,timestamp,lion_class,role,content,ln_id,sender,recipient,metadata
0,2024-10-21T13:19:35.268914,Instruction,user,{'guidance': 'Provide a summary of the informa...,ln59b379-d99-d7-c54-50086d16df88199e3e8f03,user,ln50d5f9d2ffd2-0389c-14d-1c9978f-936b71356,{'last_updated': {'recipient': 1729531177.5257...
1,2024-10-21T13:19:37.525440,AssistantResponse,assistant,"{'assistant_response': '```json { ""title"": ""...",ln8e34a63-09e9026488bee8-fb01b-b6-387ec6e4,ln50d5f9d2ffd2-0389c-14d-1c9978f-936b71356,user,{'model_response': {'id': 'chatcmpl-AKqM3Ixkns...


In [6]:
Markdown(branch.messages[0].chat_msg["content"])


---
 # Task
## **Task guidance**
Provide a summary of the information


## **Task instruction**
Use search tool to gather information on the topic, we are testing


## **Task context**
What is the meaning of life?
- respond_schema_info: {'$defs': {'ActRequestModel': {'properties': {'function': {'anyOf': [{'type': 'string'}, {'type': 'null'}], 'default': None, 'description': 'Specify the name of the function to execute. **Choose from the provided `tool_schemas`; do not invent function names.**', 'examples': ['print', 'add', 'len'], 'title': 'Function'}, 'arguments': {'default': {}, 'description': 'Provide the arguments to pass to the function as a dictionary. **Use argument names and types as specified in the `tool_schemas`; do not invent argument names.**', 'examples': [{'num1': 1, 'num2': 2}, {'x': 'hello', 'y': 'world'}], 'title': 'Arguments', 'type': 'object'}}, 'title': 'ActRequestModel', 'type': 'object'}, 'ReasonModel': {'properties': {'title': {'anyOf': [{'type': 'string'}, {'type': 'null'}], 'default': None, 'title': 'Title'}, 'content': {'anyOf': [{'type': 'string'}, {'type': 'null'}], 'default': None, 'title': 'Content'}, 'confidence_score': {'anyOf': [{'type': 'number'}, {'type': 'null'}], 'default': None, 'description': 'Provide an objective numeric confidence score between 0 and 1 (with 3 decimal places) indicating how likely you successfully achieved the task according to user expectation. Interpret the score as:\n- **1**: Very confident in a good job.\n- **0**: Not confident at all.\n- **[0.8, 1]**: You can continue the path of reasoning if needed.\n- **[0.5, 0.8)**: Recheck your reasoning and consider reverting to a previous, more confident reasoning path.\n- **[0, 0.5)**: Stop because the reasoning is starting to be off track.', 'examples': [0.821, 0.257, 0.923, 0.439], 'title': 'Confidence Score'}}, 'title': 'ReasonModel', 'type': 'object'}}, 'properties': {'title': {'anyOf': [{'type': 'string'}, {'type': 'null'}], 'default': None, 'description': 'Provide a concise title summarizing the step.', 'title': 'Title'}, 'content': {'anyOf': [{'type': 'string'}, {'type': 'null'}], 'default': None, 'description': 'Describe the content of the step in detail.', 'title': 'Content'}, 'reason': {'anyOf': [{'$ref': '#/$defs/ReasonModel'}, {'type': 'null'}], 'default': None, 'description': '**a concise reasoning for the step**', 'title': 'Reason'}, 'action_requests': {'default': [], 'description': 'List of actions to be performed if `action_required` is **True**. Leave empty if no action is required. **When providing actions, you must choose from the provided `tool_schemas`. Do not invent function or argument names.**', 'items': {'$ref': '#/$defs/ActRequestModel'}, 'title': 'Actions', 'type': 'array'}, 'action_required': {'default': False, 'description': 'Specify whether the step requires actions to be performed. If **True**, the actions in `action_requests` must be performed. If **False**, the actions in `action_requests` are optional. If no tool_schemas are provided, this field is ignored.', 'title': 'Action Required', 'type': 'boolean'}}, 'title': 'StepModelRequest', 'type': 'object'} 

- tool_schemas: {'tools': [{'type': 'function', 'function': {'name': 'mock_search', 'description': 'A mock search function that returns a string', 'parameters': {'type': 'object', 'properties': {'query': {'type': 'string', 'description': None}}, 'required': ['query']}}}]} 

None


## **Task response format**
**MUST RETURN JSON-PARSEABLE RESPONSE ENCLOSED BY JSON CODE BLOCKS.** 
```json
{'title': str | None, 'content': str | None, 'reason': lion_core.operative.reason_model.ReasonModel | None, 'action_requests': [{'function': 'str | None', 'arguments': 'dict[str, Any]'}], 'action_required': <class 'bool'>}
```




---


In [7]:
res = await branch.communicate(
    "provide 5 function calling query",
    tools=[mock_search],
    clear_messages=True,
    invoke_action=True,
    num_parse_retries=3,
)

if isinstance(res, str):
    display(Markdown(res))
else:
    print(as_readable_json(res))

Here are five function calling queries for the `mock_search` function:

1. ```json
   {
     "function": "mock_search",
     "parameters": {
       "query": "latest technology trends"
     }
   }
   ```

2. ```json
   {
     "function": "mock_search",
     "parameters": {
       "query": "best practices in software development"
     }
   }
   ```

3. ```json
   {
     "function": "mock_search",
     "parameters": {
       "query": "top tourist destinations in Europe"
     }
   }
   ```

4. ```json
   {
     "function": "mock_search",
     "parameters": {
       "query": "healthy recipes for dinner"
     }
   }
   ```

5. ```json
   {
     "function": "mock_search",
     "parameters": {
       "query": "upcoming movie releases"
     }
   }
   ```

In [8]:
branch.to_df()

,timestamp,lion_class,role,content,ln_id,sender,recipient,metadata
0,2024-10-21T13:19:37.546018,Instruction,user,{'instruction': 'provide 5 function calling qu...,lne98503-9-5b-84d7d83017c8ef-787fa439e04fb,user,ln50d5f9d2ffd2-0389c-14d-1c9978f-936b71356,{'last_updated': {'recipient': 1729531181.1101...
1,2024-10-21T13:19:41.109870,AssistantResponse,assistant,{'assistant_response': 'Here are five function...,lnffcfcb-dfe-ec85be0ddb6a465859-e2-402e256,ln50d5f9d2ffd2-0389c-14d-1c9978f-936b71356,user,{'model_response': {'id': 'chatcmpl-AKqM50dP6W...
2,2024-10-21T13:19:41.110661,ActionRequest,assistant,"{'action_request': {'function': 'mock_search',...",ln43605c0dc4a112-d3-27-d37dbfa-5d19139eb98,ln50d5f9d2ffd2-0389c-14d-1c9978f-936b71356,ln791578c6a2584-d44800-234599e-2-c8892e16e,"{'last_updated': {'sender': 1729531181.110792,..."
3,2024-10-21T13:19:41.111021,ActionRequest,assistant,"{'action_request': {'function': 'mock_search',...",lnfdb90d566-3a265d75b1-3a6db3-b3-572186888,ln50d5f9d2ffd2-0389c-14d-1c9978f-936b71356,ln791578c6a2584-d44800-234599e-2-c8892e16e,"{'last_updated': {'sender': 1729531181.111097,..."
4,2024-10-21T13:19:41.111249,ActionRequest,assistant,"{'action_request': {'function': 'mock_search',...",ln735e69e-6f13c3d-a4c693-f-81def0fa778f1af,ln50d5f9d2ffd2-0389c-14d-1c9978f-936b71356,ln791578c6a2584-d44800-234599e-2-c8892e16e,"{'last_updated': {'sender': 1729531181.111309,..."
5,2024-10-21T13:19:41.111440,ActionRequest,assistant,"{'action_request': {'function': 'mock_search',...",ln8dcf9a-1-45a64d26e8b4-c372a7b872-a2debd5,ln50d5f9d2ffd2-0389c-14d-1c9978f-936b71356,ln791578c6a2584-d44800-234599e-2-c8892e16e,"{'last_updated': {'sender': 1729531181.111495,..."
6,2024-10-21T13:19:41.111619,ActionRequest,assistant,"{'action_request': {'function': 'mock_search',...",lndad96a2-b35b4eafe7-eb-6b8-ac3219b167e966,ln50d5f9d2ffd2-0389c-14d-1c9978f-936b71356,ln791578c6a2584-d44800-234599e-2-c8892e16e,"{'last_updated': {'sender': 1729531181.111674,..."
7,2024-10-21T13:19:41.112480,ActionResponse,assistant,{'action_request_id': 'ln43605c0dc4a112-d3-27-...,ln450b8141-1e74-9d7e6646-7cac6d1-c6025dde5,ln791578c6a2584-d44800-234599e-2-c8892e16e,ln50d5f9d2ffd2-0389c-14d-1c9978f-936b71356,{'last_updated': {'content': 1729531181.112928...
8,2024-10-21T13:19:41.113367,ActionResponse,assistant,{'action_request_id': 'lnfdb90d566-3a265d75b1-...,lnda9de4-59fb3457f8-ab3af5-340-0f03f181ced,ln791578c6a2584-d44800-234599e-2-c8892e16e,ln50d5f9d2ffd2-0389c-14d-1c9978f-936b71356,{'last_updated': {'content': 1729531181.113679...
9,2024-10-21T13:19:41.113927,ActionResponse,assistant,{'action_request_id': 'ln735e69e-6f13c3d-a4c69...,ln3784430b289-2a02e-86cc-c9-1e5d4fbd9cb7f0,ln791578c6a2584-d44800-234599e-2-c8892e16e,ln50d5f9d2ffd2-0389c-14d-1c9978f-936b71356,{'last_updated': {'content': 1729531181.114174...
